In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

!pwd

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content


In [3]:
import os
os.chdir('/content/drive/My Drive/isic/')
!ls

ae_datagenerator.py	     model.py	    test.ipynb		    utils.py
c2ae.py			     models	    train_c2ae.ipynb
classifier_datagenerator.py  __pycache__    train_classifier.ipynb
data			     pyimagesearch  train_classifier.py


In [4]:
!pip install q keras==2.1.5
%tensorflow_version 1.x
import keras
keras.__version__

     |████████████████████████████████| 337kB 9.9MB/s 
  Found existing installation: Keras 2.2.5
    Uninstalling Keras-2.2.5:
      Successfully uninstalled Keras-2.2.5


Using TensorFlow backend.


'2.1.5'

In [5]:
!pip install efficientnet

In [6]:
from ae_datagenerator import autoencoder_generator
















Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




In [0]:
from c2ae import build_c2ae
import tensorflow as tf
from utils import LATENT_DIM,EFNB4_INPUT_SHAPE,EFNB2_INPUT_SHAPE,\
                        DENSENET121_INPUT_SHAPE
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint,EarlyStopping,\
                            LearningRateScheduler,CSVLogger,LambdaCallback
import numpy as np

In [0]:
from pyimagesearch.learningratefinder import LearningRateFinder
from pyimagesearch.clr_callback import CyclicLR

In [0]:
def l1_loss(y_true, y_pred):
    return tf.reduce_mean(tf.abs(y_true-y_pred))

def c2ae_loss(alpha=0.9): #condition_type
    #condition_type -> Matching/Non Matching
    #If matching, then 1, else 0.
    def custom_l1_loss(y_true,y_pred):
        #return condition_type
        
        match_image = y_true[:,:,:3]
        nonmatch_image = y_true[:,:,3:]
        
        match_recon = y_pred[:,:,:3]
        nonmatch_recon = y_pred[:,:,3:]
        
        return alpha*l1_loss(match_image,match_recon) + \
                (1-alpha)*l1_loss(nonmatch_image,nonmatch_recon)
        
    return custom_l1_loss

In [10]:
c2ae = build_c2ae('densenet121')[0]
c2ae.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 8)            0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 8)            0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 1024)         0                                            
__________________________________________________________________________________________________
H_gamma (Model)                 (None, 1024)         265472      input_4[0][0]                    
                                                                 input_5[0][0]                    
__________

In [0]:
c2ae.compile(optimizer=Adam(0.0001),loss=c2ae_loss(0.65))

In [0]:
subsample=0.1
batch_size = 32
train_nb = np.ceil(20264*subsample/batch_size)
val_nb = np.ceil(5067*subsample/batch_size)

In [0]:
train_data_gen = autoencoder_generator(batch_size,
                                     DENSENET121_INPUT_SHAPE[:-1],
                                     'densenet121',
                                     'train',
                                     subsample,
                                     nonmatch_random=False)

In [0]:
lrf = LearningRateFinder(c2ae)
lrf.find(
    trainData=train_data_gen,
    startLR=1e-6, endLR=1e-1,
    useGen=True,
    epochs=3,
    stepsPerEpoch=np.ceil((20264*subsample/float(batch_size))),
    batchSize=batch_size,
    verbose=1)

Epoch 1/3
17/64 [======>.......................] - ETA: 15:04 - loss: 0.2404

In [0]:
lrf.plot_loss()

In [0]:
subsample=1

In [0]:
batch_size = 32
train_nb = np.ceil(20264*subsample/batch_size)
val_nb = np.ceil(5067*subsample/batch_size)

In [0]:
train_data_gen = autoencoder_generator(batch_size,
                                     DENSENET121_INPUT_SHAPE[:-1],
                                     'densenet121',
                                     'train',
                                     subsample,
                                     nonmatch_random=False)

val_data_gen = autoencoder_generator(batch_size,
                                     DENSENET121_INPUT_SHAPE[:-1],
                                     'densenet121',
                                     'val',
                                     subsample,
                                     nonmatch_random=False)

In [0]:
num_epochs = 50

In [0]:
MIN_LR = 5e-5
MAX_LR = 1e-2
STEP_SIZE = 8*train_nb
CLR_METHOD = "triangular2"
clr = CyclicLR(
	mode=CLR_METHOD,
	base_lr=MIN_LR,
	max_lr=MAX_LR,
	step_size=STEP_SIZE)

In [0]:
def scheduler(epoch,lr):
    if epoch>0 and epoch%5==0:
        return lr*0.5
    return lr

In [0]:
filepath="models/c2ae/c2ae-v2-save-{epoch:02d}-{loss:.3f}-{val_loss:.3f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', 
                             save_weights_only=True,
                             verbose=1, save_best_only=False, mode='min',
                             period=1)
csv_logger = CSVLogger('training_log.csv',append=True)
#earlystopping = EarlyStopping(monitor='val_loss',verbose=1,mode='min')
lr_scheduler = LearningRateScheduler(scheduler,verbose=1)
callbacks_list = [checkpoint,csv_logger,lr_scheduler]#,earlystopping,clr

In [0]:
c2ae = build_c2ae('densenet121')[0]
c2ae.compile(optimizer=Adam(1e-4),loss=c2ae_loss(0.65))

In [23]:
train_history = c2ae.fit_generator(generator=train_data_gen,
                                steps_per_epoch=train_nb,
                                epochs=num_epochs,
                                validation_data=val_data_gen,
                                validation_steps=val_nb,
                                callbacks=callbacks_list,verbose=1,
                                initial_epoch=0)

Epoch 1/50

Epoch 00001: LearningRateScheduler reducing learning rate to 9.999999747378752e-05.
634/634 [==============================] - 5880s 9s/step - loss: 0.1189 - val_loss: 0.1810

Epoch 00001: saving model to models/c2ae/c2ae-v2-save-01-0.119-0.181.hdf5
Epoch 2/50

Epoch 00002: LearningRateScheduler reducing learning rate to 9.999999747378752e-05.
634/634 [==============================] - 3734s 6s/step - loss: 0.1043 - val_loss: 0.1803

Epoch 00002: saving model to models/c2ae/c2ae-v2-save-02-0.104-0.180.hdf5
Epoch 3/50

Epoch 00003: LearningRateScheduler reducing learning rate to 9.999999747378752e-05.
634/634 [==============================] - 3336s 5s/step - loss: 0.1027 - val_loss: 0.1807

Epoch 00003: saving model to models/c2ae/c2ae-v2-save-03-0.103-0.181.hdf5
Epoch 4/50

Epoch 00004: LearningRateScheduler reducing learning rate to 9.999999747378752e-05.
634/634 [==============================] - 2972s 5s/step - loss: 0.1027 - val_loss: 0.1812

Epoch 00004: saving model 

KeyboardInterrupt: ignored